In [ ]:
import os
os.environ['THEANORC'] = './theanorc_cpu'

import torch
import torch.nn as nn
import theano
import numpy as np
import theano.tensor as T

print (torch.__version__, theano.__version__)

In [ ]:
from ops import pytorch_wrapper

# Simple example

In [ ]:
dtype = torch.FloatTensor

# We use neural networks for testing, but you can use any function from pytorch
net = nn.Sequential(nn.Conv2d(3,4,5)).type(dtype)
x = np.random.rand(2,3,13,13).astype(np.float32)

#### Get output and grad in pytorch

In [ ]:
# Forward
input = torch.autograd.Variable(dtype(x), requires_grad=True)
out_var = net(input).sum()

# Backward
out_var.backward()

input_grad_pytorch = input.grad.data.numpy()
out_pytorch = out_var.data.numpy()

#### Now try to get the same values using theano

In [ ]:
# Wrap forward pass
f = pytorch_wrapper(net, dtype=dtype, debug=True)

# Create theano graph
xt = T.tensor4('x')
yt = f(xt).sum()
gy = T.grad(yt, xt)

# Define function
f_grad = theano.function([xt], gy, on_unused_input='warn')
f_out = theano.function([xt], yt, on_unused_input='warn')

In [ ]:
out_theano = f_out(x)

In [ ]:
input_grad_theano = f_grad(x)

In [ ]:
assert np.isclose(out_theano, out_pytorch), 'Outputs do not match'
assert np.all(np.isclose(input_grad_theano, input_grad_pytorch)), 'Grads do not match'

# Test 2

AIS but with dummy network

In [ ]:
import h5py
f = h5py.File('/sdh/data_fuel/mnist.hdf5')

num_test = 4
num_samples= 2

X = f['features'].value[60000:]

permutation = np.random.RandomState(seed=2919).permutation(X.shape[0])
X= X[permutation][:num_test]

In [ ]:
dtype = torch.cuda.FloatTensor
net = nn.Sequential(nn.ConvTranspose2d(10,1,32))
net(torch.autograd.Variable(torch.zeros(2,10,1,1))).size()

In [ ]:
class NetWrapper(nn.Module):
    '''
        The AIS code needs samples to be of shape (B x 32^2)
        But my network produces (B x 32 x 32)
        So we use this wrapper.
    '''
    def __init__(self, net):
        super(NetWrapper, self).__init__()
        self.net = net
        
    def forward(self, x):
        return self.net(x.view(-1,10,1,1)).view(-1,32*32) / 2. + 0.5
    
    
def generator(z):
    '''
        This function should define theano computational graph
        for evaluating net(z), for `z` -- latent vector of shape [B x Z_dim]
    '''
    
    f = pytorch_wrapper(NetWrapper(net),dtype=dtype)
    out = f(z)
    
    return out

import sys
sys.path.append("./eval_gen/")

from sampling import samplers_32 as samplers
lld, pf,finalstate = samplers.run_ais(generator, 
                                      X, 
                                      num_samples,
                                      num_steps=10000, 
                                      sigma=0.03, 
                                      hdim=10, 
                                      L=10, 
                                      epsilon=0.01, 
                                      data='continuous',
                                      prior="normal")